<a href="https://colab.research.google.com/github/versiware/Data_Project_Default_analysis/blob/criar-nova-pasta/Analise_de_Inadimplencia_de_clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SD MAP and Beam Search: Análise de inadimplência de clientes**

*Importando Bibliotecas*

In [ ]:
!pip install pysubgroup
import pandas as pd
import pysubgroup as ps
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 2.0 MB/s eta 0:00:00


*Loading de Dados*

In [ ]:
data_path = '/content/CC GENERAL.csv'

df = pd.read_csv(data_path)
print("Shape inicial:", df.shape)

Shape inicial: (8950, 18)


*Substituindo valores nulos pela mediana da coluna*

In [ ]:
df = df.fillna(df.median(numeric_only=True))

*Encontrando subgrupos onde os valores médios de BALANCE sejam significativamente diferentes (mais altos ou mais baixos) do valor médio global*

In [ ]:
target = ps.NumericTarget('BALANCE')
search_space = ps.create_selectors(df, ignore=['CUST_ID', 'BALANCE'])
quality_function = ps.StandardQFNumeric(a=0.5)

*Criando a Tarefa de Subgrupos*

In [ ]:
task = ps.SubgroupDiscoveryTask(
    df,
    target,
    search_space,
    result_set_size=10,
    depth=3,
    qf=quality_function
)
result = ps.BeamSearch().execute(task)

media_global = df['BALANCE'].mean()
df_results = result.to_dataframe()

print("="*60)
print("Análise Detalhada dos Subgrupos de Maior Impacto")
print("="*60)
print(f"Média Global: R$ {media_global:,.2f}")

for index, row in df_results.iterrows():
    # Coletando os valores da linha
    qualidade = row['quality']
    descricao = row['subgroup']
    tamanho_subgrupo = row['size_sg']
    media_sg = row['mean_sg']

    # Imprimindo o "card" de resultado para cada subgrupo
    print(f"Qualidade: {qualidade:.3f} | Subgrupo: {descricao}")
    print(f"   > Tamanho: {tamanho_subgrupo:,.0f} cartões")
    print(f"   > Média do Subgrupo: R$ {media_sg:,.2f}")
    print("-" * 40)

Análise Detalhada dos Subgrupos de Maior Impacto
Média Global: R$ 1,564.47
Qualidade: 139102.674 | Subgrupo: CREDIT_LIMIT>=7000.0 AND MINIMUM_PAYMENTS>=994.39
   > Tamanho: 803 cartões
   > Média do Subgrupo: R$ 6,473.30
----------------------------------------
Qualidade: 138515.067 | Subgrupo: BALANCE_FREQUENCY>=1.0 AND CREDIT_LIMIT>=7000.0 AND MINIMUM_PAYMENTS>=994.39
   > Tamanho: 740 cartões
   > Média do Subgrupo: R$ 6,656.39
----------------------------------------
Qualidade: 135067.097 | Subgrupo: CREDIT_LIMIT>=7000.0 AND MINIMUM_PAYMENTS>=994.39 AND PRC_FULL_PAYMENT: [0.0:0.08[
   > Tamanho: 708 cartões
   > Média do Subgrupo: R$ 6,640.61
----------------------------------------
Qualidade: 133420.287 | Subgrupo: CREDIT_LIMIT>=7000.0 AND MINIMUM_PAYMENTS>=994.39 AND TENURE>=12
   > Tamanho: 726 cartões
   > Média do Subgrupo: R$ 6,516.17
----------------------------------------
Qualidade: 122549.928 | Subgrupo: MINIMUM_PAYMENTS>=994.39
   > Tamanho: 1,790 cartões
   > Média do S